In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_image(image_path):
    """Load an image from the specified path and convert to grayscale."""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def preprocess_image(gray):
    """Apply Gaussian blur and enhance contrast."""
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    equalized = cv2.equalizeHist(blurred)
    return equalized

def detect_edges(image):
    """Apply Canny edge detection."""
    edges = cv2.Canny(image, 50, 150)
    return edges

def morphological_processing(edges):
    """Apply morphological operations to refine edges."""
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)
    return eroded

def measure_crack_width(binary_image, pixel_to_mm_ratio=0.1):
    """Estimate crack width in mm assuming a given pixel-to-mm ratio."""
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    widths = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        widths.append(w * pixel_to_mm_ratio)
    
    if widths:
        max_width = max(widths)
    else:
        max_width = 0
    
    return max_width

# Example workflow
def main(image_path):
    gray = load_image(image_path)
    processed = preprocess_image(gray)
    edges = detect_edges(processed)
    refined_edges = morphological_processing(edges)
    crack_width = measure_crack_width(refined_edges)
    
    # Display results
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1), plt.imshow(gray, cmap='gray'), plt.title('Grayscale')
    plt.subplot(1, 3, 2), plt.imshow(edges, cmap='gray'), plt.title('Edges')
    plt.subplot(1, 3, 3), plt.imshow(refined_edges, cmap='gray'), plt.title('Refined Edges')
    plt.show()
    
    print(f"Estimated crack width: {crack_width:.2f} mm")
    
# Example usage:
# main('path_to_concrete_image.jpg')
